# 分词器的特殊标记处理问题

ChatGPT和通义千问都无法处理回答中<|endoftext|>消失的情况。有概率触发重写，或者拼凑的情况，如，<|endoftext|>会被回复为&lt;/endoftext&gt;

为此，经过调研，我们将提供以下方案：



### 1. 字符填充法（Character Padding）

字符填充法涉及在特殊字符周围添加额外的字符（通常是空格）以避免被语言模型误解。例如，如果一个特定的上下文标记是`<|endoftext|>`，可以将其修改为`< |endoftext| >`，在标记的两端添加空格。这种方法简单且通常有效，但可能不适用于所有情况，因为它改变了原始文本的结构。

### 2. 转义字符（Escaping Characters）

转义字符方法涉及在特殊字符前添加一个或多个特定的转义符号。例如，HTML和XML使用`&lt;`和`&gt;`来代表`<`和`>`。在与大型语言模型交互时，可以定义一个转义机制，例如将特殊标记`<start>`转换为`&lt;start&gt;`。然而，这种方法要求模型能够理解转义后的格式，或者在处理文本之前和之后添加额外的步骤来转换文本。

### 3. 提示微调（Prompt Tuning）

大模型全量微调对每个任务训练一个模型，开销和部署成本都比较高。同时，离散的prompts（指人工设计prompts提示语加入到模型）方法，成本比较高，并且效果不太好。通过反向传播更新参数来学习prompts，而不是人工设计prompts；同时冻结模型原始权重，只训练prompts参数，训练完以后，用同一个模型可以做多任务推理。

### 4. 使用不同的标记

如果可能，另一个简单的解决方案是选用不同的标记，这些标记不会被模型误解。例如，如果使用`<|start|>`和`<|end|>`作为特殊标记，但它们导致问题，可以改用更独特的字符串，如`__START__`和`__END__`，或者任何不太可能与模型内部操作冲突的标记。

### 5. 分段处理

分段处理涉及将包含特殊标记的文本分成多个部分，这样标记就可以在模型处理之外单独处理。这可以通过先移除特殊标记，将文本送入模型处理，然后在模型输出中根据需要重新插入特殊标记来实现。这种方法的效果取决于特殊标记的作用和上下文的复杂性。

### 6. 自定义预处理和后处理

为模型开发自定义的预处理和后处理步骤，可以识别和处理特殊的上下文标记。这可能包括编写代码来识别特殊标记并将它们转换为模型可以理解的形式，或者将模型的输出转换回期望的格式。




# 综上所述
考虑到成本以及代码修改量，出于简单可行的目的，我们将尝试使用基于字符填充的自定义预处理和后处理。以下是两种写法，第一种更好理解，第二种有利于扩展功能

In [ ]:
import re
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        # 自定义预处理，处理特殊空字符串和未知标记，如"[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"
        text = text.replace("<|endoftext|>", "< |endoftext| >").replace("<|unk|>", "< |unk| >")
        
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # 在指定的标点符号前替换去掉空格
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        
        # 自定义后处理，还原特殊空字符串和未知标记
        text = text.replace("< |endoftext| >", "<|endoftext|>").replace("< |unk| >", "<|unk|>")
        
        return text


In [4]:
# 示例代码，构建词汇表和tokenizer实例
raw_text = "This is a sample <endoftext> text <unk> with special tokens."
preprocessed = re.split(r'([,.?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]

# 添加特殊标记
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token: integer for integer, token in enumerate(all_tokens)}
tokenizer = SimpleTokenizerV2(vocab)

# 编码和解码示例
encoded = tokenizer.encode(raw_text)
decoded = tokenizer.decode(encoded)

print("Encoded:", encoded)
print("Decoded:", decoded)


Encoded: [3, 5, 4, 6, 1, 8, 2, 10, 7, 9, 0]
Decoded: This is a sample <endoftext> text <unk> with special tokens.


- 该实现需要考虑与tokenizer的兼容性。如基于词频的BPE（Byte-Pair Encoding)，基于概率的WordPiece。也许更适合会字节回退的SentencePiece
- 如情况r"\s+"  # \s+ 匹配一个或多个空白字符（比如空格、制表符、换行符等）等，可能会出现去除空格合并的情况。此时用转义字符更为可靠

截至目前24年3月，ChatGPT与通义千问仍会在回答中出现<|endoftext|>消失的情况。
